In [1]:
import tensorflow as tf
from os import path
import sys
tf.logging.set_verbosity(tf.logging.ERROR)
sys.path.append(path.join("../.."))

In [2]:
%%html
<style>
table {
    float: left;
}

.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>

In this example, we used two datasets of pancreatic cells. For gene markers, we used the Panglao Cell Type Marker database.

We used the Baron (2016) dataset for training the neural network (NN) because it has more cells and classes, and we used the Xin (2016) dataset for testing purposes because all of its classes are present in the Baron dataset.

In [3]:
EPOCHS = 200

In [4]:
import saly
import saly.backend as S
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


SPECIES = 'Human'
DATA_PATH = '/home/mlongar/Data/SingleCellGeneExpression/'
#DATA_PATH = '/Users/MarkDavidLongar/Documents/Coding/MachineLearning/Data/'
#DATA_PATH = 'D:/Data/SingleCellGeneExpression/'
DATASET_BARON_PATH = DATA_PATH + 'baron_2016h.h5ad' 
DATASET_XIN_PATH = DATA_PATH + 'xin_2016.h5ad'
MARKER_PATH = DATA_PATH + 'panglao_gene_markers.tab.gz'

Using TensorFlow backend.


# Load the data

In [5]:
baron_data, baron_labels = saly.load_h5ad(DATASET_BARON_PATH)
xin_data, xin_labels = saly.load_h5ad(DATASET_XIN_PATH)

### Baron

In [6]:
print(baron_data.shape)
baron_labels.unique()

(8569, 20125)


[Acinar cells, Beta cells, Delta cells, PaSC, Ductal cells, Alpha cells, Other, PP cells, Endothelial cell]
Categories (9, object): [Acinar cells, Beta cells, Delta cells, PaSC, ..., Alpha cells, Other, PP cells, Endothelial cell]

Baron has 8569 cells and 20125 genes.

The cells are classified into 9 classes: Acinar cells, Alpha cells, Beta cells, Delta cells, Ductal cells, Endothelial cells, ***Other***, PaSC (i.e. Pancreatic stellate cells) and PP cells (i.e. Gamma cells).

### Xin

In [7]:
print(xin_data.shape)
xin_labels.unique()

(1492, 39851)


[Beta cells, Alpha cells, Delta cells, PP cells]
Categories (4, object): [Beta cells, Alpha cells, Delta cells, PP cells]

Xin has 1492 cells and 39851 genes.

The cells are classified into 4 classes: Alpha cells, Beta cells, Delta cells and PP cells (i.e. Gamma cells).

# Drop rows

We removed every cell classified as 'Other' from the Baron dataset.

In [8]:
baron_data, baron_labels = saly.drop_rows(baron_data, baron_labels, ['Other'])

Dropped 118 cell(s). New shape: (8451, 20125)


This removed 118 cells from Baron, leaving us with 8451 left.

# Load marker genes

We make sure to only load the Human marker genes.

In [9]:
markers_db = saly.load_markers(MARKER_PATH, SPECIES)

#### Get the markers and only keep those found in both datasets

In [10]:
markers = saly.get_mutual_markers([baron_data.columns, xin_data.columns], markers_db)

We do this because we want to train the network only on the marker genes found in both datasets, and drop the rest.

In [ ]:
print(len(S.get_used_markers(baron_data.columns, markers_db)), len(S.get_used_markers(xin_data.columns, markers_db)))
print(len(markers))

Baron has 7618 markers, and Xin has 7610.

We used only those found in both datasets (7545).

*Note that in this context* ***Marker ≠ Marker Gene*** *because multiple cell types (i.e. markers) can have the same marker gene and so in this context, the number 7545 means 7545 connections in the partially-connected Marker Layer between the cell types and the genes.*

### Make sure every class is in the marker layer

Because some cell types are differently labelled in the marker database, we created a dictionary of aliases.

We also make sure to print out any label that cannot be found in either the list of markers or in the dictionary of aliases.

In [11]:
marker_aliases = {'PaSC' : 'Pancreatic stellate cells',
                  'PP cells' : 'Gamma (PP) cells',
                  'Endothelial cell' : 'Endothelial cells'}

saly.check_labels([baron_labels, xin_labels], markers, marker_aliases)

# Drop unused genes

We used only the genes found in both datasets and the marker gene database, and therefore, we will connect with the partially-connected marker layer.

In [12]:
baron_data_ = saly.drop_unused_genes(baron_data, markers)
xin_data_ = saly.drop_unused_genes(xin_data, markers)

Dropped 15952 gene(s). New shape: (8451, 4173)
Dropped 35678 gene(s). New shape: (1492, 4173)


After this operation we are left with 4173 genes.

# Building the model

For the model we define a partially-connected Marker Layer.

The layer has a node for each cell type whose genes are present in both data sets.
In this case, there are 179 nodes in the marker layer representing 179 cell types. 

We only connect nodes (i.e. the cell types) in the marker layer to their marker genes at the input.

We constructed the partially connected marker layer by taking a dense layer and adding a connection mask (a binary matrix) to it, which we multiply with the output matrix.

We trained the network for 100 epochs.

For training the network we used two losses:

    1) A loss that measured the network's classification accuracy (cross entropy)
    
    2) A loss that measured the network's reconstruction accuracy (mean square error)
           During training we scaled the MSE score 100x to compensate for the losses' different scales

### The Architecture
|Layer|Nodes|
|-----|------|
|Input|4173 (genes)|
|Markers|179 (cell types)|
|Dense layer|100|
|**Bottleneck**|**25**|
|Dense layer|100|
|*Dropout*|*10%*|
|Output|4173 (genes)|

#### Create the model

In [22]:
model, marker_model, encoder_model = saly.build_model(baron_data_, markers, supervised=True);

# Data transformation

### Get Random Genes

In [14]:
from random import sample
gene_intersection = baron_data.columns.intersection(xin_data.columns)
random_genes = sample(gene_intersection.to_list(), baron_data_.shape[1])

In [16]:
def get_to_drop(data, to_keep):
    return [gene for gene in data.columns if gene not in to_keep]

baron_data= baron_data.drop(get_to_drop(baron_data, random_genes), axis=1)
xin_data = xin_data.drop(get_to_drop(xin_data, random_genes), axis=1)

baron_data= baron_data.reindex(sorted(baron_data.columns), axis=1)
xin_data = xin_data.reindex(sorted(xin_data.columns), axis=1)

#### Make sure the data sets' shapes match

In [17]:
saly.check_shape([baron_data, xin_data])

#### Preprocess the data

We shuffle, log10 transform and split the data into 70%, 15%, 15% splits.

In [18]:
(train_x, train_y), (validation_x, validation_y), (test_x, test_y) = saly.preprocess_data(baron_data, baron_labels,
                                                                                          train=0.7, validation=0.15,
                                                                                          test=0.15)

#### Log10 transform the Xin data, too

In [19]:
xin_data = S.log_10(xin_data)

# Train the model

In [23]:
history = saly.train_model(model, train_x, train_y, markers, marker_aliases,
                           EPOCHS, validation_data=(validation_x, validation_y), verbose=2);

Train on 5915 samples, validate on 1267 samples
Epoch 1/200
 - 3s - loss: 11.4503 - cell_activations_loss: 9.6514 - output_loss: 0.0180 - cell_activations_marker_prediction_metric: 0.0183 - val_loss: 10.9006 - val_cell_activations_loss: 9.5997 - val_output_loss: 0.0130 - val_cell_activations_marker_prediction_metric: 0.0126
Epoch 2/200
 - 2s - loss: 10.6611 - cell_activations_loss: 9.3978 - output_loss: 0.0126 - cell_activations_marker_prediction_metric: 0.0183 - val_loss: 10.5307 - val_cell_activations_loss: 9.3400 - val_output_loss: 0.0119 - val_cell_activations_marker_prediction_metric: 0.0126
Epoch 3/200
 - 3s - loss: 10.3138 - cell_activations_loss: 9.1407 - output_loss: 0.0117 - cell_activations_marker_prediction_metric: 0.0183 - val_loss: 10.2094 - val_cell_activations_loss: 9.0767 - val_output_loss: 0.0113 - val_cell_activations_marker_prediction_metric: 0.0126
Epoch 4/200
 - 2s - loss: 10.0089 - cell_activations_loss: 8.8812 - output_loss: 0.0113 - cell_activations_marker_pred

KeyboardInterrupt: 

In [21]:
saly.plot_model_history(history)
saly.test_model(model, test_x, test_y, markers, marker_aliases);

NameError: name 'history' is not defined

# Data analysis

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
colours = S.load_label_colours('../Data/baron_xin_label_colours.pickle') 

## Baron predictions

In [ ]:
baron_cell_activations = marker_model.predict(test_x)
baron_bottleneck_activations = encoder_model.predict(test_x)

In [ ]:
saly.draw_confusion_matrix(test_y, baron_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_results(test_y, baron_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_average_auc(test_y, baron_cell_activations, markers, marker_aliases)

The graph below shows the mean cell type activations. Note that only one cell type explicitly activates.

In [ ]:
saly.plot_activation_distribution(baron_cell_activations, markers,
                                  title='Average distribution of Baron cell type activations')

The network correctly classifies 98% of test data from the training database.

In [ ]:
saly.compare_embeddings([baron_cell_activations, test_y], [baron_bottleneck_activations, test_y],
                        tsne, colours=colours, alpha=0.8,
                        graph_titles=['tSNE of Baron cell type activations', 'tSNE of Baron bottleneck activations'])

## Xin predictions

In [ ]:
xin_cell_activations = marker_model.predict(xin_data)
xin_bottleneck_activations = encoder_model.predict(xin_data)

In [ ]:
saly.draw_confusion_matrix(xin_labels, xin_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_results(xin_labels, xin_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_average_auc(xin_labels, xin_cell_activations, markers, marker_aliases)

In [ ]:
saly.plot_activation_distribution(xin_cell_activations, markers,
                                  title='Average distribution of Xin cell type activations')

The network correctly classifies almost 100% of data from a completely new data set.

In [ ]:
saly.compare_embeddings([xin_cell_activations, xin_labels],
                        [xin_bottleneck_activations, xin_labels],
                        tsne, colours=colours, alpha=0.8,
                        graph_titles=['tSNE of Xin cell type activations',
                                      'tSNE of Xin bottleneck activations'])

# Reconstruction

Here we took the Xin data set and run them through the autoencoder.

Then we took the autoencoder's output (i.e. the reconstructed data) and ran that data through the autoencoder again, this time looking at the Marker layer's cell type activations.

The cell type activations of the reconstructed data give similar, sometimes worse, but almost never better results.

In [ ]:
reconstruction = model.predict(xin_data)[1]
reconstruction = pd.DataFrame(reconstruction, columns=xin_data.columns)
reconstructed_cell_activations = marker_model.predict(reconstruction)
reconstructed_bottleneck_activations = encoder_model.predict(reconstruction)

In [ ]:
saly.draw_confusion_matrix(xin_labels, reconstructed_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_results(xin_labels, reconstructed_cell_activations, markers, marker_aliases)

In [ ]:
saly.get_average_auc(xin_labels, reconstructed_cell_activations, markers, marker_aliases)

In [ ]:
saly.plot_activation_distribution(reconstructed_cell_activations, markers,
                                  title='Average distribution of reconstructed Xin cell type activations')

The network correctly classifies almost 99% of reconstructed Xin data, similar to the original data.

In [ ]:
saly.compare_embeddings([reconstructed_cell_activations, xin_labels],
                        [reconstructed_bottleneck_activations, xin_labels],
                        tsne, colours=colours, alpha=0.8,
                        graph_titles=['tSNE of reconstruced Xin cell type activations',
                                      'tSNE of reconstruced Xin bottleneck activations'])

# Combined data

Here we see if our marker activations still have batch effects.

In [ ]:
saly.draw_comparison([baron_cell_activations, test_y], [xin_cell_activations, xin_labels] ,tsne, colours=colours,
                     graph_title='Combined cell activations of Baron and Xin cell type activations')

In [ ]:
saly.draw_comparison([baron_cell_activations, test_y], [reconstructed_cell_activations, xin_labels], tsne,
                     colours=colours, 
                     graph_title='Combined cell activations of Baron and the reconstructed Xin cell type activations')

# Top 3 activations

We also tried to see if taking only the top 3 cell activations would help us remedy the batch effect.

In [ ]:
baron_top_3 = saly.get_top_activations(3, baron_cell_activations)
xin_top_3 = saly.get_top_activations(3, xin_cell_activations)
reconstructed_top_3 = saly.get_top_activations(3, reconstructed_cell_activations)

In [ ]:
saly.draw_comparison([baron_top_3, test_y], [xin_top_3, xin_labels], tsne, colours=colours,
                     graph_title='Combined data of top 3 activations of Baron and Xin')

In [ ]:
saly.draw_comparison([baron_top_3, test_y], [reconstructed_top_3, xin_labels], tsne, colours=colours,
                     graph_title='Combined data of top 3 activations of Baron and reconstruced Xin')